# PyOpenMagnetics: Getting Started Tutorial

Welcome to PyOpenMagnetics (PyMKF)! This notebook introduces the core concepts and functionality.

## What is PyOpenMagnetics?

PyOpenMagnetics is a Python wrapper for the MKF (Magnetics Knowledge Foundation) C++ engine. It follows the MAS (Magnetic Agnostic Structure) JSON schema for describing magnetic components.

**Key Features:**
- Design magnetic components (inductors, transformers, chokes)
- Calculate core and winding losses
- Query databases of cores, materials, and wires
- Optimize designs for specific applications

In [ ]:
# Install PyOpenMagnetics if needed
# !pip install pyopenmagnetics

import PyOpenMagnetics
import json
from pprint import pprint

print(f"PyOpenMagnetics loaded successfully!")

## 1. Exploring the Database

PyOpenMagnetics includes databases of:
- Core shapes (E, ETD, PQ, RM, etc.)
- Core materials (ferrites, powdered iron)
- Wires (round, litz, foil)

In [ ]:
# Get available core shapes
shapes = PyOpenMagnetics.get_core_shape_names(include_toroidal=False)
print(f"Available shapes: {len(shapes)}")
print(f"Sample shapes: {shapes[:10]}")

In [ ]:
# Get available materials
materials = PyOpenMagnetics.get_core_material_names()
print(f"Available materials: {len(materials)}")
print(f"Sample materials: {materials[:10]}")

In [ ]:
# Get wire options
wires = PyOpenMagnetics.get_wire_names()
print(f"Available wires: {len(wires)}")
print(f"Sample wires: {wires[:10]}")

## 2. Creating a Core

A core is defined by its shape and material. You can reference standard parts or create custom ones.

In [ ]:
# Create a core using standard references
# Note: 'type' is required and specifies core construction
core_data = {
    "functionalDescription": {
        "name": "My ETD 39 Core",
        "type": "two-piece set",
        "shape": "ETD 39/20/13",
        "material": "3C95",
        "gapping": [],
        "numberStacks": 1
    }
}

# Process the core to get full specifications
# API: calculate_core_data(core_data, include_material_data)
core = PyOpenMagnetics.calculate_core_data(core_data, False)
print("Core created successfully!")
print(f"Effective length: {core['processedDescription']['effectiveParameters']['effectiveLength']:.4f} m")
print(f"Effective area: {core['processedDescription']['effectiveParameters']['effectiveArea']:.6f} m²")

## 3. Core Material Properties

Get material properties for loss estimation and permeability calculations.

In [ ]:
# Get Steinmetz coefficients for core loss estimation
# The Steinmetz equation: P = k * f^α * B^β
frequency = 100000  # 100 kHz
coefficients = PyOpenMagnetics.get_core_material_steinmetz_coefficients("3C95", frequency)

print("Steinmetz Coefficients for 3C95 at 100 kHz:")
print(f"  k     = {coefficients.get('k', 'N/A')}")
print(f"  alpha = {coefficients.get('alpha', 'N/A')}")
print(f"  beta  = {coefficients.get('beta', 'N/A')}")

# Get material permeability at operating conditions
temperature = 25.0   # °C
dc_bias = 0.0        # A/m
permeability = PyOpenMagnetics.get_material_permeability("3C95", temperature, dc_bias, frequency)
print(f"\nRelative permeability: {permeability:.0f}")

## 4. Comparing Materials

Compare different core materials at various frequencies.

In [ ]:
# Compare permeability of different materials at various frequencies
try:
    import matplotlib.pyplot as plt
    
    materials = ["3C90", "3C95", "3C97"]
    frequencies = [10000, 50000, 100000, 200000, 500000]
    
    plt.figure(figsize=(10, 6))
    
    for mat in materials:
        perms = []
        for freq in frequencies:
            try:
                perm = PyOpenMagnetics.get_material_permeability(mat, 25.0, 0.0, float(freq))
                perms.append(perm)
            except:
                perms.append(None)
        
        valid_freqs = [f/1000 for f, p in zip(frequencies, perms) if p is not None]
        valid_perms = [p for p in perms if p is not None]
        
        if valid_perms:
            plt.plot(valid_freqs, valid_perms, '-o', linewidth=2, markersize=8, label=mat)
    
    plt.xlabel('Frequency (kHz)', fontsize=12)
    plt.ylabel('Relative Permeability', fontsize=12)
    plt.title('Permeability vs Frequency for Different Ferrites', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Install matplotlib for visualizations: pip install matplotlib")

## 5. Settings Configuration

Configure global settings to customize PyOpenMagnetics behavior.

In [ ]:
# Get current settings
settings = PyOpenMagnetics.get_settings()
print("Current settings:")
print(f"  - Paint style: {settings.get('painterMode', 'N/A')}")
print(f"  - Use toroidal cores: {settings.get('useOnlyManufacturerRecommendedGaps', 'N/A')}")

In [ ]:
# Reset to defaults
PyOpenMagnetics.reset_settings()
print("Settings reset to defaults.")

# Get settings after reset
settings_after = PyOpenMagnetics.get_settings()
print(f"Settings keys: {list(settings_after.keys())[:5]}...")

## Next Steps

- **02_buck_inductor.ipynb**: Design a buck converter inductor
- **03_flyback_transformer.ipynb**: Design a flyback transformer
- **04_core_adviser.ipynb**: Use the core adviser to find optimal cores
- **05_winding_losses.ipynb**: Calculate winding losses with proximity effects

## Resources

- [PyOpenMagnetics Documentation](https://github.com/OpenMagnetics/PyMKF)
- [MAS Schema](https://github.com/OpenMagnetics/MAS) - Magnetic Agnostic Structure
- [MKF Engine](https://github.com/OpenMagnetics/MKF) - Magnetics Knowledge Foundation